In [ ]:
#
# Attempts to implement the following:
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
from   udist_scatterplots_via_sectors import UDistScatterPlotsViaSectors, xyUniformSampleDistributionSectorTransformDEBUG, xyUniformSampleDistributionSectorTransform
import time
import rtsvg
rt = rtsvg.RACETrack()
import random
num_of_pts   = [100, 200, 50]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

_iterations_ = 4

x_new, y_new, svgs, svgs_for_sectors, svg_animation, df_fine = xyUniformSampleDistributionSectorTransformDEBUG(rt, _xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_)

t0 = time.time()
xyUniformSampleDistributionSectorTransform(rt, _xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_)
t1 = time.time()
udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True)
t2 = time.time()
#
# Comparison (w/ fixed implementation)
# 351 Points (128 Iterations), 18.74s Reference Time, 1.66s Polars Time | (M1 Pro 16G) 
# 351 Points (256 Iterations), 38.45s Reference Time, 7.28s Polars Time | (M1 Pro 16G) 
#
print(f'{len(_xvals_)} Points ({_iterations_} Iterations), {t1-t0:.2f}s Reference Time, {t2-t1:.2f}s Polars Time')
rt.tile([svg_animation, udspvs.animateIterations(animation_dur="4s")], spacer=10)

In [ ]:
# 2048 Iterations w/ 351 Points | M1 Pro 16G
#
#                 sector_sums | 3.959
#                 arctangents | 2.436
#   ray_segment_intersections | 2.289
#              explode_points | 2.251
#         add_missing_sectors | 1.839
#                   area_calc | 1.254
#         sector_uv_summation | 0.907
#                point_update | 0.784
#       prepare_sector_angles | 0.659
#          join_sector_angles | 0.592
#                 all_sectors | 0.421
#                   normalize | 0.269
#                  prepare_df | 0.000
udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=2048)
_lu_ = {'subroutine':[], 'time':[]}
for k in udspvs.time_lu: _lu_['subroutine'].append(k), _lu_['time'].append(udspvs.time_lu[k])
_df_ = pl.DataFrame(_lu_).sort(by='time', descending=True)
for i in range(len(_df_['subroutine'])): print(f'{_df_["subroutine"][i]:>28} | {_df_["time"][i]:.3f}')